<a href="https://colab.research.google.com/github/seph0916/logistic-Regression/blob/main/%ED%98%BC%EA%B3%B5%EB%A8%B8%EC%8B%A05%EB%8B%A8%EC%9B%90_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# 트리의 앙상블
# 앙상블 학습-> 정형데이터를 다루기에 가장 뛰어난 성과를 내는 알고리즘이다.
# 비정형 데이터를 다루기에는 신경망 알고리즘이 괜찮음.

In [17]:
# 앙상블 학습의 대표주자인 랜덤 포레스트
# 랜덤포레스트 -> 트리를 훈련하기위해 데이터를 추출하게되는데 중복추출이 될수도 있다 ,기본적으로 100개의 트리를 훈련
# 이렇게 만들어진 샘플을 부트스트랩 샘플 이라고 부름 , 기본적으로 부트스트랩 샘플과 훈련세트의 크기는 같게 함
# 또한 랜덤포레스트는 랜덤하게 선택한 샘플과 특성을 사용하기 떄문에 훈련세트 과적합되는걸 막아주며 안정적 성능을 얻을수 있다.

In [1]:
# input 데이터와 target 데이터를 나누기 위해 구분
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
wine=pd.read_csv('https://bit.ly/wine-date')
data=wine[['alcohol','sugar','pH']].to_numpy()
target=wine[['class']].to_numpy()
train_input,test_input,train_target,test_target=train_test_split(data,target,test_size=0.2,random_state=42)

In [2]:
from sklearn.model_selection import cross_validate
from sklearn.ensemble import RandomForestClassifier
rf=RandomForestClassifier(n_jobs=-1,random_state=42)

In [3]:
# return_train_score =True -> 검증점수 뿐 아니라 훈련점수도 반환하게 해주는 매개변수
scores=cross_validate(rf,train_input,train_target,return_train_score=True,n_jobs=-1)
print(np.mean(scores['train_score']),np.mean(scores['test_score']))

0.9973541965122431 0.8905151032797809


In [4]:
# 결정 트리의 큰 장점 -> 특성 중요도를 계산함, 특성중요도는 각 결정 트리의 특성 중요도를 취합한것.
# 각각 알코올 , 당도 ,pH 순 -> 1절 결정 트리에서 만든 특성 중요도와 다름
# 당도의 중요도가 상당히 높았으나 지금은 낮아진상태-> 이는 하나의 특성에대해 과도하게 적합되려는것을 방지하기 위해 특성일부를 랜덤하게 선택하여 훈련
rf.fit(train_input,train_target)
print(rf.feature_importances_)

<ipython-input-4-d16175f611b7>:4: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  rf.fit(train_input,train_target)


[0.23167441 0.50039841 0.26792718]


In [5]:
# 랜덤 포레스트분류는 자체적으로 모델을 평가하는 점수를 얻을수 있다.
# 부트 스트랩 샘플에 포함되지 않고 남는 샘플이 있는데 이를 OOB(out of bag)샘플이라고 함 이 남는 샘플을 사용하여 부트스트랩 샘플로 훈련한 트리를 평가할수있음
# 마치 검증세트와 같은 역할
# 매개변수 oob_score=True 설정하면 됨

In [6]:
rf=RandomForestClassifier(oob_score=True,n_jobs=-1,random_state=42)
rf.fit(train_input,train_target)

<ipython-input-6-e256c5636b53>:2: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  rf.fit(train_input,train_target)


RandomForestClassifier(n_jobs=-1, oob_score=True, random_state=42)

In [7]:
print(rf.oob_score_)

0.8934000384837406


In [8]:
# 엑스트라 트리 랜덤 포레스트와 비슷함, 차이점: 부트스트랩 샘플을 사용하지 않음
# 즉 전체 훈련세트를 사용 DecisionTreeClassifier에서 splitter 매개변수를 random으로 지정. 하는 결정트리임.
# 가장 좋은 분할을 찾는것이 아니라 무작위로 분할함-> 무작위 분할은 성능이 낮아지겠지만 과대적합을 막고 검증세트의 점수를 높이는 효과가 있다.

In [9]:
# extra모델의 교차 검증점수 확인
from sklearn.ensemble import ExtraTreesClassifier
et=ExtraTreesClassifier(n_jobs=-1,random_state=42)
scores=cross_validate(et,train_input,train_target,return_train_score=True,n_jobs=-1)
print(np.mean(scores['train_score']),np.mean(scores['test_score']))

0.9974503966084433 0.8887848893166506


In [10]:
scores

{'fit_time': array([1.44125962, 1.19810104, 1.27805662, 1.21949387, 0.5866034 ]),
 'score_time': array([0.20276189, 0.14363766, 0.10503483, 0.07572937, 0.07576442]),
 'test_score': array([0.88365385, 0.87884615, 0.90375361, 0.88835419, 0.88931665]),
 'train_score': array([0.9971133 , 0.99663219, 0.998076  , 0.997595  , 0.9978355 ])}

In [11]:
et.fit(train_input,train_target)
print(et.feature_importances_)

<ipython-input-11-d33477e215d5>:1: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  et.fit(train_input,train_target)


[0.20183568 0.52242907 0.27573525]


In [12]:
# 그레이디언트 부스팅
# 깊이가 얕은 결정 트리를 사용하여 이전 트리의 오차를 보완하는 방식, 기본적으로 깊이3, 결정트리100개 사용
# 얕은 결정트리를 사용하기 떄문에 과대적합에 강함, 경사하강법 사용

In [13]:
from sklearn.ensemble import GradientBoostingClassifier
gb=GradientBoostingClassifier(random_state=42)
scores=cross_validate(gb,train_input,train_target,return_train_score=True,n_jobs=-1)
print(np.mean(scores['train_score']),np.mean(scores['test_score']))

0.8881086892152563 0.8720430147331015


In [15]:
# 그레이디언트 부스팅 분류는 트리 개수를 늘려도 과대적합에 강함
# 학습률, 트리 갯수 증가 시켜보기
gb=GradientBoostingClassifier(n_estimators=500,learning_rate=0.2,random_state=42)
scores=cross_validate(gb,train_input,train_target,return_train_score=True,n_jobs=-1)
print(np.mean(scores['train_score']),np.mean(scores['test_score']))

0.9464595437171814 0.8780082549788999


In [16]:
gb.fit(train_input,train_target)
print(gb.feature_importances_)

/usr/local/lib/python3.10/dist-packages/sklearn/ensemble/_gb.py:437: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


[0.15872278 0.68010884 0.16116839]


In [17]:
# 트리 훈련에 사용할 훈련세트의 비율을 정하는 subsample -> 기본값은 1.0으로 전체훈련세트 사용
# 일반적으로 그레이디언트 부스팅이 랜덤포레스트보다 조금 더 높은 성능을 얻음 하지만 트리를 추가하면 훈련속도 느려짐 -> n_jobs 매개변수가 없다.

In [21]:
# 히스토그램 기반 그레디언트 부스팅: 정형 데이터를 다루는 머신러닝 알고리즘.
# 입력 특성을 256개의 구간으로 나눈다. 노드를 분할할때 최적의 분할을 매우 빠르게 찾을수 있음.
from sklearn.experimental import enable_hist_gradient_boosting
from sklearn.ensemble import HistGradientBoostingClassifier
hgb=HistGradientBoostingClassifier(random_state=42)
scores=cross_validate(hgb,train_input,train_target,return_train_score=True)
print(np.mean(scores['train_score']),np.mean(scores['test_score']))

/usr/local/lib/python3.10/dist-packages/sklearn/utils/validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.10/dist-packages/sklearn/utils/validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.10/dist-packages/sklearn/utils/validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.10/dist-packages/sklearn/utils/validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for exam

0.9321723946453317 0.8801241948619236


In [22]:
# 히스토그램 그래디언트 부스팅의 특성중요도 계산 함수 permutation_importance()
from sklearn.inspection import permutation_importance
hgb.fit(train_input,train_target)
result=permutation_importance(hgb,train_input,train_target,n_repeats=10,random_state=42,n_jobs=-1)
print(result.importances_mean)

/usr/local/lib/python3.10/dist-packages/sklearn/utils/validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


[0.08876275 0.23438522 0.08027708]


In [25]:
# test 세트에서의 특성중요도 확인, 테스트 세트 성능 확인
result=permutation_importance(hgb,test_input,test_target,n_repeats=10,random_state=42,n_jobs=-1)
print(result.importances_mean)
print(hgb.score(test_input,test_target))

[0.05969231 0.20238462 0.049     ]
0.8723076923076923


In [28]:
# xgboost -> 히스토그램 기반 그레이디언트 부스팅 라이브러리
from xgboost import XGBClassifier
xgb=XGBClassifier(tree_method='hist',random_state=42)
scores=cross_validate(xgb,train_input,train_target,return_train_score=True)
print(np.mean(scores['train_score']),np.mean(scores['test_score']))

0.9555033709953124 0.8799326275264677


In [29]:
# lightgbm -> 히스토그램 기반 그레이디언트 부스팅 라이브러리
from lightgbm import LGBMClassifier
lgb=LGBMClassifier(random_state=42)
scores=cross_validate(lgb,train_input,train_target,return_train_score=True,n_jobs=-1)

In [30]:
print(np.mean(scores['train_score']),np.mean(scores['test_score']))

0.935828414851749 0.8801251203079884
